# Fashion (MACS 40400: Computation and the Identification of Cultural Patterns)
## Part II: Working with APIs

Today, we're going to attempt to discover if there are any conventional interpretants for Zegna's Uomo cologne. In class on Tuesday, we saw that the product was intended to produce interpretants related to the concepts of masculinity, mastery and sophistication. Today, we are evaluating whether these are the sorts of interpretants that are actually produced in the circles of interpreters outside of individual advertisement that we watched by analyzing the words used in YouTube videos and the things that appear in images related to the Zegna brand more generally on Flickr.

## Ermenegildo Zegna Image Object Detection
### Flickr and Google Cloud API Demonstration

One way we might begin to understand some of the interpretants produced from the Uomo sign vehicle is to identify the objects that are in images and videos related to the Zegna brand as a whole. You might remember that the advertisement we watched on Tuesday begins with the phrase "Ermenegildo Zegna Presents..." and this Zegna name seems to be a key part in the labeling of this product. What meaning does this brand name bestow upon the product? We might begin to understand the interpretant(s) of "Ermenegildo Zegna" by identifying the entities that commonly occur in photos related to the name.

We will use the Flickr API and a pre-trained computer vision algorithm with Google Cloud Vision's API that detects objects in images related to the term "Ermenegildo Zegna" to try to identify general patterns in the way in which photographers on Flickr compose their photos related to the Zegna brand (professional shots of models, as well as from the general public). Note that in this scenario, we are not only using an API to get pre-existing data (Flickr), but also to process data for us (Google Cloud Vision).

It takes a bit too long to run this code as an in-class exercise (and requires getting an API key for the Flickr API, or some other image sharing website, along with a Google Cloud API key), but I'd encourage you to play around with and expand upon this approach on your own and/or work with image/video data in this way in your final projects.

If you want to use [Google Cloud Vision API](https://cloud.google.com/vision/), you will need to sign up for a Google Cloud account, get an API key, as well run the following command on your command line to run the code below:
`pip install google-cloud`. For more information on the approach below, see Google's official [documentation](https://cloud.google.com/vision/docs/labels#vision-label-detection-python).

In [50]:
import pandas as pd
import requests
import flickrapi
from google.cloud import vision
import io

First, let us use the Flickr API to identify images related to the keywords "Ermengildo Zegna." Here, for demonstration purposes, we will collect 500 total images, but we could collect more. We will collect a list of the image URLs.

In [4]:
flickr = flickrapi.FlickrAPI('##############################', '#######################', cache=True)

keyword = 'Ermenegildo Zegna'
urls=[]
for i, photo in enumerate(flickr.walk(text=keyword, sort='relevance', per_page=500, tags=keyword, extras='url_c')):
    url = photo.get('url_c')
    urls.append(url)
    
    # Get 500 urls:
    if i > 500:
        break

Then, we can get each image via the Requests library and use Google Cloud Vision's API to label each image with text of what is in it.

In [19]:
def detect_labels(img_file):
    """Detects labels for an image file input."""
    client = vision.ImageAnnotatorClient()

    image = vision.types.Image(content=img_file)

    response = client.label_detection(image=image)
    labels = response.label_annotations

    return [label.description for label in labels]

In [20]:
labels_flickr = []

for url in urls:
    try:
        response = requests.get(str(url))
        img = response.content
        detected_labels = detect_labels(img)
        labels_flickr.extend(detected_labels)
    
    except:
        pass

Finally, we can see some of the labels most associated with the Ermengildo Zegna images. It appears words like "gentleman," "urban/city/building," "car/vehicle," "male," etc. all appear fairly prominently, reinforcing the "classic," masculine image of the brand we saw in the advertisement and it seems that the photos are generally associated with something of an "urban gentleman" -- a general interpretant of these Ermengildo Zegna images.

In [24]:
pd.DataFrame({'labels':labels_flickr, 
              'frame':range(len(labels_flickr))}
            ).groupby('labels').count().sort_values('frame', ascending=False)[:50]

,frame
labels,
Architecture,75
Building,72
Outerwear,44
Suit,36
Formal wear,34
City,34
Gentleman,33
Facade,33
Photography,32


Note that using other image sources (such as Twitter, Instagram, or a company website) might enhance this brief study with additional data about how photographic interpretants for the brand vary within different communities of interpreters.

If we wanted to detect custom entities that are outside the scope of Google's pre-trained algorithms, we could also train our own machine learning algorithms. For instance, in my own work, I've trained classifiers to identify archaeological sites from vast amounts of satellite image data (using Python's OpenCV and Scikit learn packages). This involves feeding a classification algorithm a lot of images with labels of what is in them, so that it can begin to learn for itself how they are used. This is about as deep as we will get into these approaches in this class, but I'd encourage you to explore these approaches in your final projects if you'd like to work with images.

*******

## Importing YouTube Video Captions and Metadata

To identify the interpretants for the Uomo cologne in particular, we will gather data from YouTube videos that review the product. What language do the YouTube influencers use to describe the cologne? How do they generally judge the product? Is it consistent with the argument presented by the Uomo advertisers in the advertisement we watched on Tuesday?

To gather YouTube metadata and captions, we can use the [official YouTube API](https://developers.google.com/youtube/v3/docs/search/list) (for which you can acquire keys through the same portal as Google Cloud Vision). However, I find it significantly easier in Python to use the third-party scraper [youtube_dl](https://github.com/ytdl-org/youtube-dl/blob/master/README.md#readme). The advantage of using `youtube_dl` is that it provides a framework for downloading video data on other streaming sites as well (for instance, Vimeo and TikTok) and returns JSON formatted data in the same way as the official YouTube API. One disadvantage, however, is that it does not return comment data from YouTube. You will need to use the official YouTube API if you want to collect comments for videos.

In [1]:
#Install youtube-dl for downloading youtube videos/metadata (as well as information from other video streaming sites)
!pip install youtube_dl

In [2]:
import youtube_dl
import re

Once we have installed youtube_dl, we can return all metadata related to 'Zegna Uomo' videos as well as any available captions (automatically generated, or otherwise).

In [3]:
ydl_opts = {'dump_single_json': True, 'writeautomaticsub': True, 'subtitleslangs': ['en']}
with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    result = ydl.extract_info("ytsearch100:Zegna Uomo", download=False)

[youtube:search] query "Zegna Uomo": Downloading page 1
[youtube:search] query "Zegna Uomo": Downloading page 2
[youtube:search] query "Zegna Uomo": Downloading page 3
[youtube:search] query "Zegna Uomo": Downloading page 4
[youtube:search] query "Zegna Uomo": Downloading page 5
[download] Downloading playlist: Zegna Uomo
[youtube:search] playlist Zegna Uomo: Collected 100 video ids (downloading 100 of them)
[download] Downloading video 1 of 100
[youtube] Otqi2zZrRJM: Downloading webpage
[youtube] Otqi2zZrRJM: Downloading video info webpage
[youtube] Otqi2zZrRJM: Looking for automatic captions
[download] Downloading video 2 of 100
[youtube] 4cWpCynRQzw: Downloading webpage
[youtube] 4cWpCynRQzw: Downloading video info webpage
[youtube] 4cWpCynRQzw: Looking for automatic captions
[download] Downloading video 3 of 100
[youtube] axzIdt6rFYM: Downloading webpage
[youtube] axzIdt6rFYM: Downloading video info webpage
[youtube] axzIdt6rFYM: Looking for automatic captions
[youtube] axzIdt6rFYM

[download] Downloading video 12 of 100
[youtube] TFKz6QDY_9k: Downloading webpage
[youtube] TFKz6QDY_9k: Downloading video info webpage
[youtube] TFKz6QDY_9k: Looking for automatic captions
[download] Downloading video 13 of 100
[youtube] 6jeZgHWlsao: Downloading webpage
[youtube] 6jeZgHWlsao: Downloading video info webpage
[youtube] 6jeZgHWlsao: Looking for automatic captions
[download] Downloading video 14 of 100
[youtube] e1BxYRif2FM: Downloading webpage
[youtube] e1BxYRif2FM: Downloading video info webpage
[youtube] e1BxYRif2FM: Looking for automatic captions
[download] Downloading video 15 of 100
[youtube] 0uzaSR-9owA: Downloading webpage
[youtube] 0uzaSR-9owA: Downloading video info webpage
[youtube] 0uzaSR-9owA: Looking for automatic captions
[youtube] 0uzaSR-9owA: Downloading MPD manifest
[download] Downloading video 16 of 100
[youtube] 51iCVeAYHro: Downloading webpage
[youtube] 51iCVeAYHro: Downloading video info webpage
[youtube] 51iCVeAYHro: Looking for automatic captions
[d

[download] Downloading video 21 of 100
[youtube] myGkJ3fJjrQ: Downloading webpage
[youtube] myGkJ3fJjrQ: Downloading video info webpage
[youtube] myGkJ3fJjrQ: Looking for automatic captions
[download] Downloading video 22 of 100
[youtube] tgZis3DkQVo: Downloading webpage
[youtube] tgZis3DkQVo: Downloading video info webpage
[youtube] tgZis3DkQVo: Looking for automatic captions
[download] Downloading video 23 of 100
[youtube] ioSGxWHiaZU: Downloading webpage
[youtube] ioSGxWHiaZU: Downloading video info webpage
[youtube] ioSGxWHiaZU: Looking for automatic captions
[download] Downloading video 24 of 100
[youtube] QbkoRoZZ4gw: Downloading webpage
[youtube] QbkoRoZZ4gw: Downloading video info webpage
[youtube] QbkoRoZZ4gw: Looking for automatic captions


[download] Downloading video 25 of 100
[youtube] YKx4BfZ2fJQ: Downloading webpage
[youtube] YKx4BfZ2fJQ: Downloading video info webpage
[youtube] YKx4BfZ2fJQ: Looking for automatic captions
[download] Downloading video 26 of 100
[youtube] d1_R6gFj_pw: Downloading webpage
[youtube] d1_R6gFj_pw: Downloading video info webpage
[youtube] d1_R6gFj_pw: Looking for automatic captions


[download] Downloading video 27 of 100
[youtube] 6jeZgHWlsao: Downloading webpage
[youtube] 6jeZgHWlsao: Downloading video info webpage
[youtube] 6jeZgHWlsao: Looking for automatic captions
[download] Downloading video 28 of 100
[youtube] UW-zHgizU3I: Downloading webpage
[youtube] UW-zHgizU3I: Downloading video info webpage
[youtube] UW-zHgizU3I: Looking for automatic captions


...

[download] Downloading video 95 of 100
[youtube] Gq3LiSFx1t4: Downloading webpage
[youtube] Gq3LiSFx1t4: Downloading video info webpage
[youtube] Gq3LiSFx1t4: Looking for automatic captions


[youtube] Gq3LiSFx1t4: Downloading MPD manifest
[download] Downloading video 96 of 100
[youtube] 9dem-ZU9OLE: Downloading webpage
[youtube] 9dem-ZU9OLE: Downloading video info webpage
[youtube] 9dem-ZU9OLE: Looking for automatic captions
[youtube] 9dem-ZU9OLE: Downloading MPD manifest
[download] Downloading video 97 of 100
[youtube] DieAckaG7TY: Downloading webpage
[youtube] DieAckaG7TY: Downloading video info webpage
[youtube] DieAckaG7TY: Looking for automatic captions


[download] Downloading video 98 of 100
[youtube] DsthYpCm0XM: Downloading webpage
[youtube] DsthYpCm0XM: Downloading video info webpage
[youtube] DsthYpCm0XM: Looking for automatic captions


[download] Downloading video 99 of 100
[youtube] Aq_QxwNf5N8: Downloading webpage
[youtube] Aq_QxwNf5N8: Downloading video info webpage
[youtube] Aq_QxwNf5N8: Looking for automatic captions


[download] Downloading video 100 of 100
[youtube] qNlo5ZuOdpA: Downloading webpage
[youtube] qNlo5ZuOdpA: Downloading video info webpage
[youtube] qNlo5ZuOdpA: Looking for automatic captions
[download] Finished downloading playlist: Zegna Uomo


The text within the captions is quite messy, so before we load the data into a dataframe, we need to clean out special URLs, special characters, etc.:

In [47]:
tag_re = re.compile(r'<[^>]+>')
link_re = re.compile(r'http\S+|\d+')
def remove_tags(text):
    return tag_re.sub(' ', text)

def remove_links(text):
    return link_re.sub(' ', text)


title, description, tags, captions = ([] for i in range(4))

for i in result['entries']:
    # append each item to a separate list and then add them as a dictionary to dataframe all at once after the loop has run
    title.append(i['title'])
    description.append(remove_links(i['description']).replace('\n',' '))
    tags.append(remove_links(' '.join(i['tags'])))
    if 'en' in i['automatic_captions']:
        auto_captions = requests.get(i['automatic_captions']['en'][0]['url'])
        auto_captions_clean = remove_tags(auto_captions.text).replace('&amp;#39;', "'")         \
                                                             .replace('&lt;font'," ")           \
                                                             .replace('color=&quot;', " ")      \
                                                             .replace('&lt;/font&gt;', " ")     \
                                                             .replace('#CCCCCC&quot;&gt;', " ") \
                                                             .replace('#E5E5E5&quot;&gt;', " ")
        captions.append(auto_captions_clean)
    else:
        captions.append(float('nan'))
uomo_yt_df = pd.DataFrame({'Title': title, 'Description': description, 'Tags': tags, 'Captions': captions})

# uomo_yt_df.to_json('uomo_yt100.json') # save dataframe to JSON

uomo_yt_df.head()

,Title,Description,Tags,Captions
0,Uomo Ermenegildo Zegna for Men (2013),a fragrance i TRULY love Frarantica Link Fo...,zegna uomo fragrance review mens all year roun...,hey guys how are y'all doing I've been mea...
1,Redolessence Smells & Rates best Zegna fragrance,Facebook Twitter IG Redolessence Zeg...,best Zegna fragrance best Zegna cologne Zegna ...,what's going on guys so today's video I go...
2,Ermenegildo Zegna Uomo (Review) 🔥🔥Cheapie sexi...,,,hey everybody welcome back good ev...
3,Uomo by Ermenegildo Zegna - A Quicky Review,Fragrantica Profile: Zegna Website:,uomo Ermenegildo Zegna (Organization) fresh be...,come to another fragrance quickie and toda...
4,FIRST IMPRESSION! | UOMO by Ermenegildo Zegna,HELLO EVERYONE! Today at StudioScents I'm pre...,Fragrancepreview UOMO ErmenegildoZegna Firstim...,hi and welcome back to studio since I'm To...


After cleaning, we have clean text for the Title, Description, Tags, and Captions for each YouTube video related to the Zegna Uomo cologne and we've saved our data in a JSON format so that we can easily reload the data in its current form as a Pandas DataFrame. Now, we're ready for text analysis! Let's move on to Part III.